# Squeeze-and-Excitation (SE) Fusion을 해볼거임.

핵심 : "채널별로 중요도를 매겨서 강약 조절을 한다"


작동 원리:
두 Feature를 합친(Concat) 후, 신경망이 **"이 채널(Feature map)은 중요하고, 저 채널은 덜 중요하다"**는 가중치(Weight)를 계산해 곱해줍니다.
공간(Spatial) 정보는 잠시 무시하고, 특징(Feature)의 종류에 집중합니다.

장점:
가성비 최고: 연산량 증가가 거의 없는데 성능 향상은 확실한 편입니다.
구현이 매우 쉽습니다.

단점:
공간 정보 무시: 영상의 '어느 위치'가 중요한지는 고려하지 않고, 이미지 전체에 대해 채널 중요도만 따집니다.


추천 상황: "기존 학습 속도를 유지하면서, 안정적인 성능 향상을 원할 때"




# 변경한 코드

##### 1. models/residual_transformers.py에서 아래 코드 추가
위치: `ART_block` 클래스 바로 위에 추가

```python
class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block"""
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)  # Global Average Pooling
        self.excitation = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        B, C, H, W = x.size()
        # Squeeze: [B, C, H, W] → [B, C]
        y = self.squeeze(x).view(B, C)
        # Excitation: [B, C] → [B, C]
        y = self.excitation(y).view(B, C, 1, 1)
        # Scale: element-wise multiplication
        return x * y.expand_as(x)
```

---

##### 2. models/residual_transformers.py에서 아래 코드 추가
위치: `self.cc = channel_compression(...)` 바로 아래에 추가

```python 
# 이 코드를 찾아서
        self.cc = channel_compression(...) 바로 아래에 추가
    # Residual CNN
```

아래와 같이 수정함.

```python
        self.cc = channel_compression(ngf * 8, ngf * 4)
        # SE Block 추가
        self.se = SEBlock(channels=ngf * 8, reduction=16)
    # Residual CNN
```

---

##### 3. ART_block.forward 수정
위치:  `torch.cat` 바로 아래
```python
# 이 코드를 찾아서
        # concat transformer output and resnet output
        x = torch.cat([transformer_out, x], dim=1)
        # channel compression
        x = self.cc(x)
```

아래와 같이 수정함.

```python

    # concat transformer output and resnet output
        x = torch.cat([transformer_out, x], dim=1)
        # SE 적용
        x = self.se(x)
        # channel compression
        x = self.cc(x)


---
---
---

# SE 적용 전체 흐름
##### 적용 위치
**ResViT 전체 구조:**
```python
encoder_1 → encoder_2 → encoder_3 → [ART_1] → ART_2 → ... → ART_9 → decoder
                                       ↑
                                    SE 적용 (transformer가 있는 ART block만)
```
ResViT에서 ART_1과 ART_6만 transformer를 사용함. 따라서 SE는 이 두 블록에서만 작동함.

---

### ART_block 내부 흐름 (transformer 있는 경우)

```python
입력 x [B, 256, 64, 64]  ← encoder_3 출력 (CNN feature)
         │
         ├──────────────────────────────┐
         │                              │
         ▼                              │
    downsample                          │
    [B, 256, 64, 64] → [B, 1024, 16, 16]│
         │                              │
         ▼                              │
    embeddings (patch화)                 │
    [B, 1024, 16, 16] → [B, 256, 768]   │
         │                              │
         ▼                              │
    transformer encoder                 │
    [B, 256, 768] → [B, 256, 768]       │
         │                              │
         ▼                              │
    reshape                             │
    [B, 256, 768] → [B, 768, 16, 16]    │
         │                              │
         ▼                              │
    upsample                            │
    [B, 768, 16, 16] → [B, 256, 64, 64] │
         │                              │
         ▼                              ▼
    ┌────────────────────────────────────┐
    │  concat                            │
    │  Transformer출력 + CNN입력          │
    │  [B, 256, 64, 64] + [B, 256, 64, 64]│
    │  = [B, 512, 64, 64]                │
    └────────────────────────────────────┘
                    │
                    ▼
    ┌────────────────────────────────────┐
    │  ★ SE Block (새로 추가)            │
    │  [B, 512, 64, 64] → [B, 512, 64, 64]│
    └────────────────────────────────────┘
                    │
                    ▼
    ┌────────────────────────────────────┐
    │  channel_compression               │
    │  [B, 512, 64, 64] → [B, 256, 64, 64]│
    └────────────────────────────────────┘
                    │
                    ▼
    ┌────────────────────────────────────┐
    │  residual_cnn                      │
    │  [B, 256, 64, 64] → [B, 256, 64, 64]│
    └────────────────────────────────────┘
                    │
                    ▼
              출력 [B, 256, 64, 64]
```
---

### SE Block 내부 동작

```python
입력: concat된 feature [B, 512, 64, 64]
      ├─ 채널 0~255: Transformer 출력
      └─ 채널 256~511: CNN 입력

Step 1: Squeeze (Global Average Pooling)
        [B, 512, 64, 64] → [B, 512, 1, 1] → [B, 512]
        각 채널을 하나의 숫자로 요약

Step 2: Excitation (FC layers)
        [B, 512] → FC → [B, 32] → ReLU → FC → [B, 512] → Sigmoid
        채널별 중요도 점수 (0~1) 계산
        
        예: [0.9, 0.8, 0.2, 0.7, ...]
            채널0 중요, 채널2 덜 중요

Step 3: Scale (곱하기)
        [B, 512, 64, 64] × [B, 512, 1, 1]
        각 채널에 해당 점수를 곱함

출력: 재조정된 feature [B, 512, 64, 64]
```

---

### 실제 효과 예시

```python
이미지 A (뼈가 두드러진 CT):
- Transformer 채널들: 평균 점수 0.85 (강조됨)
- CNN 채널들: 평균 점수 0.60

이미지 B (연조직 위주 CT):
- Transformer 채널들: 평균 점수 0.55
- CNN 채널들: 평균 점수 0.80 (강조됨)

→ 이미지마다 Transformer/CNN 비중이 자동 조절됨
```